In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import mlflow
import mlflow.catboost          
import mlflow.sklearn
from pathlib import Path
from mlflow.entities import ViewType
from mlflow.exceptions import MlflowException
from dotenv import load_dotenv
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score,
    confusion_matrix, log_loss
)


# === 0) окружение из .env (если есть) ===
load_dotenv()

EXPERIMENT_NAME='test_connection_experiment_vadim_shakula'
RUN_NAME='baseline_logreg_v1'
REGISTRY_MODEL_NAME='churn_model_vadimshakula'

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("MLFLOW_S3_ENDPOINT_URL", "https://storage.yandexcloud.net")
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID", "")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY", "")

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# === 1) Данные и модель (ДО infer_signature!) ===
# Подставь свои пути к тестовым данным, если отличаются
X_test_path = "data/processed/X_test.csv"
y_test_path = "data/processed/y_test.csv"

X_test = pd.read_csv(X_test_path)
y_test = pd.read_csv(y_test_path).iloc[:, 0]

# Загрузка модели (если у тебя уже есть переменная model — этот блок можно пропустить)
# Подставь свой путь, если нужен другой
model_path_candidates = [
    "models_1sprint/model.pkl",
    "models/model.pkl",
]
model_path = next((p for p in model_path_candidates if os.path.exists(p)), None)
assert model_path is not None, f"Не найден файл модели. Ожидал один из: {model_path_candidates}"
model = joblib.load(model_path)

# Предсказания
prediction = model.predict(X_test)
# Вероятности положительного класса (на случай CatBoost/Sklearn)
if hasattr(model, "predict_proba"):
    proba = model.predict_proba(X_test)[:, 1]
else:
    # reserve: если у модели нет predict_proba — попытка через decision_function
    if hasattr(model, "decision_function"):
        z = model.decision_function(X_test)
        proba = 1 / (1 + np.exp(-z))
    else:
        # если вообще нет ни одного метода вероятностей — как fallback:
        proba = prediction.astype(float)

# === 2) Всё, что ниже — как у тебя, но теперь объекты определены ===
pip_requirements = "requirements.txt"
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {"model_type": "monthly"}

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
assert experiment is not None, f"Эксперимент '{EXPERIMENT_NAME}' не найден. Создай его заранее или проверь имя."
experiment_id = experiment.experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    auc = roc_auc_score(y_test, proba)
    precision = precision_score(y_test, prediction)
    recall = recall_score(y_test, prediction)
    f1 = f1_score(y_test, prediction)
    ll = log_loss(y_test, proba)
    tn, fp, fn, tp = confusion_matrix(y_test, prediction).ravel()

    metrics = {
        "roc_auc": float(auc),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
        "logloss": float(ll),
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp),
    }
    mlflow.log_metrics(metrics)

    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature=signature,
        input_example=input_example,
        pip_requirements=pip_requirements,
        await_registration_for=60,
        metadata=metadata,
    )



/home/mle-user/.local/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Registered model 'churn_model_vadimshakula' already exists. Creating a new version of this model...
2025/08/31 13:14:32 INFO mlflow.tracking._model_registry.client: Waiting u

In [14]:
client = mlflow.MlflowClient()

models = client.search_model_versions(filter_string=f"name = '{REGISTRY_MODEL_NAME}'")
print(f"Model info:\n {models}")

model_name_1 = models[-1].name
model_version_1 = models[-1].version
model_stage_1 = models[-1].current_stage

model_name_2 = models[-2].name
model_version_2 = models[-2].version
model_stage_2 = models[-2].current_stage


print(f"Текущий stage модели 1: {model_stage_1}")
print(f"Текущий stage модели 2: {model_stage_2}")

# поменяйте статус каждой модели
client.transition_model_version_stage(model_name_1, model_version_1, 'production')

client.transition_model_version_stage(model_name_2, model_version_2, 'staging')

# переимнуйте модель в реестре
client.rename_registered_model(name=REGISTRY_MODEL_NAME, new_name=f'{REGISTRY_MODEL_NAME}_b2c')

Model info:
 []


IndexError: list index out of range